# Install library

In [2]:
!pip install rdflib
!pip install PyDrive

     |████████████████████████████████| 235kB 4.1MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 


# Import library


In [3]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import Namespace, NamespaceManager
from rdflib.plugins import sparql
from rdflib.namespace import RDF, RDFS
from rdflib.serializer import Serializer

In [17]:
graph = Graph()
graph.parse("onto_xml.owl", format="application/rdf+xml")
graph.serialize(format='n3')

b'@prefix : <http://www.semanticweb.org/vasilii/ontologies/2021/3/book_onto#> .\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n:Bad a owl:Class ;\n    rdfs:subClassOf :Status .\n\n:Good a owl:Class ;\n    rdfs:subClassOf :Status .\n\n:Satisfactory a owl:Class ;\n    rdfs:subClassOf :Status .\n\n:hasAuthor a owl:InverseFunctionalProperty,\n        owl:ObjectProperty ;\n    rdfs:domain :Educational_book,\n        :Entertainment_book ;\n    rdfs:range :Author ;\n    rdfs:subPropertyOf owl:topObjectProperty .\n\n:hasDescription a owl:DatatypeProperty ;\n    rdfs:domain :Educational_book,\n        :Entertainment_book ;\n    rdfs:range xsd:string ;\n    rdfs:subPropertyOf owl:topDataProperty .\n\n:hasFavorites a owl:DatatypeProperty ;\n    rdfs:domain :Educational_book,\n        :Entertainment_book ;\n    rdfs:range xs

# Class

In [18]:
path = 'http://www.semanticweb.org/vasilii/ontologies/2021/3/book_onto#'

# Class
TAGS = URIRef(path + 'Tags')
BOOK = URIRef(path + 'Book')

STATUS_BAD = URIRef(path + 'Bad')
STATUS_GOOD = URIRef(path + 'Good')
STATUS_SATISFACTORY = URIRef(path + 'Satisfactory')
GENRE = URIRef(path + 'Genre')
AUTHOR = URIRef(path + 'Author')
EDUCATIONAL_BOOK = URIRef(path + 'Educational_book')
ENTERTAIMENT_BOOK = URIRef(path + 'Entertainment_book')

NAME_GENRES = {
    'Фэнтези': URIRef(path + 'Fentezi'),
    'ЛитРПГ': URIRef(path + 'Litrpg'),
    'Любовные романы': URIRef(path + 'Lyubovnye_romany'),
    'Фантастика': URIRef(path + 'Fantastika'),
    'Исторический роман': URIRef(path + 'Istoricheskiy_roman'),
    'Молодежная проза': URIRef(path + 'Molodejnaya_proza'),
    'Попаданцы': URIRef(path + 'Popadancy'),
    'Женский роман': URIRef(path + 'Jenskaya_proza'),
    'Фанфик': URIRef(path + 'Fanfik'),
    'Детективы': URIRef(path + 'Detektivy'),
    'Проза': URIRef(path + 'Proza'),
    'Боевик': URIRef(path + 'Boevik'),
    'Триллеры': URIRef(path + 'Trillery'),
    'Мистика/Ужасы': URIRef(path + 'Mistikaujasy'),
    'Разное': URIRef(path + 'Raznoe'),
}

# Object Property

In [45]:
# Object Property
HAS_STATUS = URIRef(path + 'hasStatus')
HAS_AUTHOR = URIRef(path + 'hasAuthor')
HAS_GENRE = URIRef(path + 'hasGenre')
HAS_TAGS = URIRef(path + 'hasTags')

# Data Property

In [20]:
# Data Property
HAS_RATE = URIRef(path + 'hasRate')
HAS_FAVORITES = URIRef(path + 'hasFavorites')
HAS_VIEWS = URIRef(path + 'hasViews')
HAS_NAME = URIRef(path + 'hasName')
HAS_URL = URIRef(path + 'hasURL')
HAS_NUMBER_OF_PAGES = URIRef(path + 'hasNumberOfPages')
HAS_DESCRIPTION = URIRef(path + 'hasDescription')
HAS_DATE_OF_PUBLICATION = URIRef(path + 'hasDateOfPublication')

# Loads data

## Get data from JSON file


In [21]:
import json

def get_data_json(path: str):
  with open(path, 'r') as f:
    data = json.load(f)
    return data

data = get_data_json('data.json')
data[0]

{'author': [{'author_name': 'Лира Алая',
   'author_page': 'https://litnet.com/ru/lira-alaya-u530349'}],
 'book_tags': ['бытовое фэнтези',
  'неунывающая героиня',
  'говорящая таблеточная машина'],
 'book_url': 'https://litnet.com/ru/book/tabletochku-vashe-temneishestvo-b333552',
 'description': '\nТаблеточку, Ваше Темнейшество?\nЛира Алая\n                Прислонилась к таблеточной машине и попала в другой мир? Не теряйся! Провизор ты или кто? Зря латынь учила, что ли? Сойдёшь за ведьму! Аптеку откроешь.\r\nСветлые тут, ко... подробнее\nВ тексте есть:  бытовое фэнтези, неунывающая героиня, говорящая таблеточная машина\n\n\n\ue86a\n                        В процессе: 15 Мая\n                        55 стр\n\n\n278441\n19292\n\n\n',
 'favourites': 19292,
 'genre': 'Фэнтези',
 'number_pages': 55,
 'rating': 2330,
 'title': 'Таблеточку, Ваше Темнейшество?',
 'type_book': 'Entertainment_book',
 'views': 278441}

## Add data in graph

In [46]:
from rdflib import Literal, XSD
import re

def replace_space(text):
  text = re.findall(r'[\w]+', text)
  return '_'.join(text).rstrip()


def add_author(authors: list, book_uri):
  for author in authors:
    author_name = URIRef(path + replace_space(author['author_name']))

    graph.add((author_name, RDF.type, AUTHOR))
    graph.add((author_name, HAS_AUTHOR, book_uri))
    graph.add((author_name, HAS_URL, Literal(author['author_page'])))
    
# num_genre = 0

def add_genre(genre: str, book_uri):
  graph.add((book_uri, HAS_GENRE, NAME_GENRES[genre]))
  graph.add((NAME_GENRES[genre], HAS_NAME, Literal(genre, datatype=XSD.string)))


def add_views(views: int, book_uri):
  graph.add((book_uri, HAS_VIEWS, Literal(views, datatype=XSD.integer)))


def add_number_pages(cnt_page: int, book_uri):
  graph.add((book_uri, HAS_NUMBER_OF_PAGES, Literal(cnt_page, datatype=XSD.integer)))


def add_favorites(favorites: int, book_uri):
  graph.add((book_uri, HAS_FAVORITES, Literal(favorites, datatype=XSD.integer)))


def add_description(desc: str, book_uri):
  graph.add((book_uri, HAS_DESCRIPTION, Literal(desc, datatype=XSD.string)))


def add_status(rate: int, book_uri):
  if rate > 5000:
    graph.add((book_uri, HAS_RATE, Literal(rate, datatype=XSD.integer)))
    graph.add((book_uri, RDF.type, STATUS_GOOD))
    graph.add((STATUS_GOOD, HAS_STATUS, book_uri))

  elif rate > 1000 and rate <= 5000:
    graph.add((book_uri, HAS_RATE, Literal(rate, datatype=XSD.integer)))
    graph.add((book_uri, RDF.type, STATUS_SATISFACTORY))
    graph.add((STATUS_SATISFACTORY, HAS_STATUS, book_uri))
  else:
    graph.add((book_uri, HAS_RATE, Literal(rate, datatype=XSD.integer)))
    graph.add((book_uri, RDF.type, STATUS_BAD))
    graph.add((STATUS_BAD, HAS_STATUS, book_uri))



def add_book_tags(tags: list, book_uri):
  for tag in tags:
    tag = URIRef(path + replace_space(tag))
    graph.add((tag, RDF.type, TAGS))
    graph.add((tag, HAS_NAME, Literal(tag, datatype=XSD.string)))
    graph.add((tag, HAS_TAGS, book_uri))


def add_book(book: dict):
  book_uri = URIRef(path + replace_space(book['title']))

  if book['type_book'] == 'Entertainment_book':
    graph.add((book_uri, RDF.type, ENTERTAIMENT_BOOK))
  else:
    graph.add((book_uri, RDF.type, EDUCATIONAL_BOOK))

  graph.add((book_uri, HAS_NAME, Literal(book['title'], datatype=XSD.string)))
  graph.add((book_uri, HAS_URL, Literal(book['book_url'], datatype=XSD.string)))

  add_status(book['rating'], book_uri)
  add_number_pages(book['number_pages'], book_uri)
  add_author(book['author'], book_uri)
  add_book_tags(book['book_tags'], book_uri)
  add_genre(book['genre'], book_uri)
  add_views(book['views'], book_uri)
  add_favorites(book['favourites'], book_uri)
  add_description(book['description'], book_uri)


In [47]:
# add_book(data[0])
for el in data:
  add_book(el)

## Save data in .owl file

In [48]:
graph.serialize("book_onto3.owl", format="turtle")

# Query in ontology

In [24]:
# Топ 10 популярных книг 
qv1= graph.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX path: <http://www.semanticweb.org/vasilii/ontologies/2021/3/book_onto#>
  SELECT ?y ?r
    WHERE {
      ?b rdf:type path:Entertainment_book .
      ?b path:hasRate ?r .
      ?b path:hasName ?y
      FILTER (?r > 20000)
    } LIMIT 10
""")
for row in qv1:
    print("%s %s" % row)

Зачем я ему? 27010
Невеста из другого мира 23015
Селфи на балконе 23621
Меняю на нового ... или обмен по-русски 41673
Дракон - не подарок. Королевская академия Драко 31173
Невеста на неделю, или Моя навеки 22353
Адвокат Зверя 23601
Кикимора для хама 28989
Взрывная смесь  29425
Маленькое чудо для злодея 28704


In [53]:
# У каких книг жанра X количество просмотров превышает Y?
qv2= graph.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX path: <http://www.semanticweb.org/vasilii/ontologies/2021/3/book_onto#>
  SELECT ?y ?v
  WHERE {
    ?b rdf:type path:Entertainment_book .
    ?b path:hasGenre path:Boevik .
    ?b path:hasViews ?v . 
    ?b path:hasName ?y
    FILTER (?v > 500000)
  }""")
for row in qv2:
    print("%s %s" % row)

Новое Начало. Книга Вторая 502248
Задача выжить 725034
Серый кукловод. Часть 1 712620
Новое Начало. Книга Первая 792966


In [52]:
# Наиболее часто добавляемые книги в библиотеку жанра боевик?
qv3= graph.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX path: <http://www.semanticweb.org/vasilii/ontologies/2021/3/book_onto#>
  SELECT ?y ?f
  WHERE {
    ?b rdf:type path:Entertainment_book .
    ?b path:hasGenre path:Boevik .
    ?b path:hasFavorites ?f . 
    ?b path:hasName ?y
  }
  ORDER BY DESC(?f)
  LIMIT 10
  """)
for row in qv3:
    print("%s %s" % row)

Поварёшка 13040
Новое Начало. Книга Первая 11300
Серый кукловод. Часть 1 9534
Новое Начало. Книга Вторая 8835
Новое Начало. Книга Третья 7507
Колонизация 6469
Механист 5622
Задача выжить 4462
Арсанты. Дети богов. 4067
Падший 3629


In [60]:
# Какие книги статуса X чаще все добавляют в свою коллекцию?
qv4= graph.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX path: <http://www.semanticweb.org/vasilii/ontologies/2021/3/book_onto#>
  SELECT ?y ?f
  WHERE {
    ?b rdf:type path:Entertainment_book .
    ?b rdf:type path:Good .
    ?b path:hasFavorites ?f . 
    ?b path:hasName ?y
  } 
  ORDER BY DESC(?f)
  LIMIT 10
  """)

for row in qv4:
    print("%s %s" % row)

Опасный босс 80126
Мой лунный эльф, или Как не влюбиться по уши 59539
Взрывная смесь  58884
Меняю на нового ... или обмен по-русски 58524
Драконий подарок. Королевская академия Драко 57307
Попаданка в Академии драконов 49651
Невеста из другого мира 49081
Пленница лунного эльфа 45390
Смертельный способ выйти замуж 42686
Маленькое чудо для злодея 42621


# Generate documentation

In [10]:
!pip install pylode

     |████████████████████████████████| 143kB 4.0MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
  Created wheel for rdflib-jsonld: filename=rdflib_jsonld-0.5.0-py2.py3-none-any.whl size=15348 sha256=12f790eb1bfdbfed463776385bb2a470653db2a04ab3796b36cf4d4c33966c5c
  Stored in directory: /root/.cache/pip/wheels/8d/e4/7f/9ebcb3e400c694e645c3adba40ef3e9bda78384ac3b9b0d13d
Successfully built rdflib-jsonld


In [16]:
import pylode

html = pylode.MakeDocco(input_data_file='onto_xml.owl', outputformat="html").document()

f = open('doc.html', 'w')
f.write(html)
f.close()